In [0]:
!pip install --upgrade tensorflow
!pip install transformers
!pip install keras
!pip install pandas
!pip install numpy
# !pip install -q pyyaml h5py

import pandas as pd
import numpy as np
import re
import keras
import numpy as np
import tensorflow as tf
import math

from sklearn.utils import shuffle

# from keras.layers import LSTM, GRU
# from keras.layers.embeddings import Embedding
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Model
from keras.layers import Input, Dense, Bidirectional,Flatten
# from keras.layers import concatenate,Dropout
from transformers import BertTokenizer, TFBertModel,TFBertForSequenceClassification

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.2.0)
     |████████████████████████████████| 665kB 4.7MB/s 
     |████████████████████████████████| 3.8MB 46.9MB/s 
     |████████████████████████████████| 1.1MB 47.6MB/s 
     |████████████████████████████████| 890kB 60.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=5e81f675ecc8967fe7ecab7a41996583ade4afadd1bbb7facf63c86c3d49aa28
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Using TensorFlow backend.


In [0]:

from google.colab import drive
drive.mount("/content/drive")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Given a natural language query $q$ and a set of (predicate, object) tuples pertaining to an entity, the model aims to predict the *importance* and *utility* ranks of the (predicate, object) tuples with respect to the query $q$.

We encode the query $q$ and the predicate $p$ into $d$-dimensional embedding vectors $h_q$ and $h_p$ using a BiLSTM. Then, the model predicts a score for by using a dot product of $h_q$ and $h_p$. i.e. $\Psi(q, p) = h_q^Th_p$

We minimize the MSE Loss between the predicted score and the target utility or importance score.

In [0]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # self.bert_layer = TFBertModel.from_pretrained('bert-base-uncased')
        self.dense_layer = tf.keras.layers.Dense(1, activation="sigmoid")
        ## 50 x 768

    def __call__(self, x_train):
        # intermediate = self.bert_layer(x_train)
        output = self.dense_layer(x_train[1])
        
        
        # input()
        return output

In [0]:
## homogeneous encoding for all the queries,predicates, objects
maxLengthPadding = 50
# Note : maxLengthOfTokens = 48;

input_dictionary = {}

# df = pd.read_csv("/content/drive/My Drive/Independent Study/Modified_Data.csv")
# df = pd.read_csv("/content/drive/My Drive/Independent Study/URI_only.csv")


## final datasets
# df = pd.read_csv("/content/drive/My Drive/Independent Study/Complete_Data.csv")
df = pd.read_csv("/content/drive/My Drive/Independent Study/URI_with_imp.csv")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

## create the position id and segment id

def getBertParameters(tokens):
    attn_mask = []
    seg_ids = []
    # pos_ids = []
    if(len(tokens)<maxLengthPadding):
        attn_mask = [1]*len(tokens) + [0]*(maxLengthPadding-len(tokens))
    else:
        attn_mask = [1]*maxLengthPadding
    
    segment = 0
    for x in tokens:
        seg_ids.append(segment)
        if(x=='[SEP]'):
            segment = 1
    seg_ids+=[0]*(maxLengthPadding-len(tokens))
    return attn_mask,seg_ids


In [0]:
## defining custom model parameters as per tensorflow2 guidelines

model = MyModel()

loss_obj = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam( learning_rate=0.00005)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanSquaredError(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanSquaredError(name='test_accuracy')

In [0]:
## group data by query

groupedDataset = df.groupby('query')
queries = []
for q,g in groupedDataset:
  queries.append(q)

queries = shuffle(queries)


In [0]:

data_length = len(groupedDataset)
split_length = int(0.8*data_length)

queries_for_training = queries[:split_length]
queries_for_testing = queries[split_length:]

test_data =[]
count = 0

queryTrainList = []
predTrainList = []
objTrainList = []

y_train = []

for query,group in groupedDataset:
  if(query in queries_for_testing):
    test_data.append([query,group["pred"].values,group["obj"].values,group["rel"].values])
  else: 
    for i in range(len(group["pred"].values)):
      queryTrainList.append(query)
      predTrainList.append(group["pred"].values[i])
      objTrainList.append(group["obj"].values[i])
      y_train.append(group["rel"].values[i])
  count+=1

## To give a higher importance to a higher utility value
def invertRanks(rankArray):
  maximum = 2 
  for i in range(len(rankArray)):
    rankArray[i]=rankArray[i]/maximum
  return rankArray


y_train = invertRanks(y_train)



queryTrainList,predTrainList,objTrainList,y_train = shuffle(queryTrainList,predTrainList,objTrainList,y_train)


print(y_train[0:10])    
print(len(queryTrainList),len(predTrainList),len(objTrainList),len(test_data),len(y_train))


[0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
997 997 997 19 997


In [0]:
print(944/16
      )

59.0


In [0]:
batch_size = 16
num_batches = math.floor(len(queryTrainList)/batch_size)
EPOCHS = 10

bert_layer = TFBertModel.from_pretrained('bert-base-uncased')


def train_step(query,pred,obj,target):
    input_ids = []
    attn_mask = []
    segment_ids = []

    for k in range(len(query)):
          in_id = tokenizer.encode(query[k],pred[k]+" "+obj[k], add_special_tokens=True)  # Batch size 1
          # print(query[k])
          # print(pred[k])
          # print(obj[k])
          # print(in_id)
          # input()
          attn,seg = getBertParameters(tokenizer.convert_ids_to_tokens(in_id))
          in_id += [0]*(maxLengthPadding-len(in_id))
          input_ids.append(in_id)
          attn_mask.append(attn)
          segment_ids.append(seg)
          
    loss = 0
    with tf.GradientTape() as tape:
        input_dictionary['input_ids'] = tf.convert_to_tensor(np.array(input_ids)) ## [None, :]
        input_dictionary['attention_mask'] = tf.convert_to_tensor(np.array(attn_mask))  ## [None, :]
        input_dictionary['token_type_ids'] = tf.convert_to_tensor(np.array(segment_ids))  ## [None, :]
        output = bert_layer(input_dictionary)
        output = model(output)
        loss = (target-output)**2
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return loss
  
        
    

## [CLS] + [Query] + [SEP] + [PRED + OBJ] +[SEP]
for j in range(EPOCHS):
    for i in range(num_batches-1):
        index = i*batch_size
        q = queryTrainList[index:index+batch_size]
        p = predTrainList[index:index+batch_size]
        o = objTrainList[index:index+batch_size]
        actual_y = np.array(y_train[index:index+batch_size]).reshape(batch_size,1)
        loss = train_step(q,p,o,actual_y)
        q = queryTrainList[num_batches*batch_size:]
        p = predTrainList[num_batches*batch_size:]
        o = objTrainList[num_batches*batch_size:]
        actual_y = y_train[num_batches*batch_size:]
        actual_y = np.array(actual_y).reshape(len(actual_y),1)
    if(len(q)>1):  
    
        loss = train_step(q,p,o,actual_y)
    # print("Loss after epoch "+str(j)+" :", loss)   
    n5,n10 = test_step()
    print(n5,n10)
    


0.4110386160510052 0.5083036143547622
0.41434326528693805 0.5208195199305636
0.45730443876928445 0.5513072666492504
0.4459550088033007 0.5482176575886468
0.4647234966589734 0.5608954150372455
0.4616105626330766 0.5585798823232837
0.46392498349881445 0.5631546265299179
0.45908369695788076 0.5584295364896438
0.46543579417984726 0.5628844836195648
0.4710748919242081 0.5655042989393879


In [0]:
# Bert w/o fine tuning on rel for URI data

0.48075428383371943 0.5528401726926597
0.48657867716690933 0.5946955715174369
0.4166653936829592 0.5105340440227891
0.436182790122136 0.5328894087396685
0.4616105626330766 0.5585798823232837


# avg :   .4563      .5498

In [0]:

# model_2.save_weights('/content/drive/My Drive/Independent Study/MiniBatch_URI_Data')
# # model_2.save_weights('/content/drive/My Drive/Independent Study/MiniBatch_Entire_Data_Imp')
# model_load = MyModel2(vocab_size)
# model_load.load_weights('/content/drive/My Drive/Independent Study/MiniBatch_URI_Data')

In [0]:
  # ## grouping the dataset by query to get the ndcg scores
def test_step():
    count = 0
    integerValuedQueryRanks = []
    groundTruthRanks = []
    for x_test in test_data:
        query = x_test[0]
        predPerQuery = list(x_test[1])
        objPerQuery = list(x_test[2])
        target = list(x_test[3])
        g_t = []
        i_ranks = []
        predictedRanksPerQuery = []
        for i in range(len(predPerQuery)):
            input_ids = tokenizer.encode(query,predPerQuery[i]+" "+objPerQuery[i], add_special_tokens=True)  # Batch size 1
            attn_mask,segment_id = getBertParameters(tokenizer.convert_ids_to_tokens(input_ids))
            input_ids += [0]*(maxLengthPadding-len(input_ids))
            input_dictionary['input_ids'] = tf.constant(input_ids)[None, :]
            input_dictionary['attention_mask'] = tf.constant(attn_mask)[None, :]
            input_dictionary['token_type_ids'] = tf.constant(segment_id)[None, :]
            output = bert_layer(input_dictionary)
            output = model(output)
            # predictedRanksPerQuery.append(output.numpy().tolist()[0][0])
            i_ranks.append(round(output.numpy().tolist()[0][0]*2))
        integerValuedQueryRanks.append(i_ranks)

        groundTruthRanks.append(target) 
        # predictedQueryRanks.append(predictedRanksPerQuery)
        count+=1
    return ndcg_scores(groundTruthRanks,integerValuedQueryRanks)

# for x,y in zip(groundTruthRanks,integerValuedQueryRanks):
#     print("GRound truth : ", x)
#     print("Predicted : " ,y)
#     print("----------------------------------------------------------x-----------------------------------------x---------------------------")
    

In [0]:

from sklearn.metrics import ndcg_score, dcg_score 
def ndcg_scores(groundTruthRanks,integerValuedQueryRanks):  
  ndcg_scores_5 = []
  ndcg_scores_10 = []
  count = 0
  for x,y in zip(groundTruthRanks,integerValuedQueryRanks):
      
      
      if(len(x)>1):
          true_relevance = np.asarray([x]) 
          relevance_score = np.asarray([y]) 

          ndcg_scores_5.append(ndcg_score(true_relevance, relevance_score,k=5))
          ndcg_scores_10.append(ndcg_score(true_relevance, relevance_score,k=10))
  

  n5 = sum(ndcg_scores_5)/len(ndcg_scores_5)
  n10 = sum(ndcg_scores_10)/len(ndcg_scores_10)

  return n5,n10

   
 

In [0]:
# NDCG@5: 0.7981107576995108
# NDCG@10: 0.8366214915585496

# for whithout fine tuning bert - partial dataset train test split

# NDCG@5: 0.6234769111104137
# NDCG@10: 0.6933181533971675

# for whithout fine tuning bert - complete dataset train test split
